In [14]:
### 필요 라이브러리
import os
import pandas as pd
import numpy as np
import datetime as dt
import math
import warnings
# warnings.filterwarnings(action='ignore')

import json
from typing import Optional
from numpy import load
from torch.utils.data import Dataset
import torch
import json
import os
import re
import threading
from os import makedirs, path, remove
import datetime
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import seaborn as sns
import pandas as pd

from tst import Transformer
from tst.loss import OZELoss

from src.dataset import OzeDataset
from src.utils import compute_loss
from src.visualization import map_plot_function, plot_values_distribution, plot_error_distribution, plot_errors_threshold, plot_visual_sample
from lxml import html
from tqdm import tqdm

ModuleNotFoundError: No module named 'tst'

In [2]:
BATCH_SIZE=4


In [3]:
# load array
x = load('xdata.npy')
y = load('ydata.npy')
# print the array


In [4]:

class LOBDataset(Dataset):
    """Torch dataset for Oze datachallenge.

    Load dataset from a single npz file.

    Attributes
    ----------
    labels: :py:class:`dict`
        Ordered labels list for R, Z and X.

    Parameters
    ---------
    dataset_path:
        Path to the dataset inputs as npz.
    labels_path:
        Path to the labels, divided in R, Z and X, in json format.
        Default is "labels.json".
    normalize:
        Data normalization method, one of ``'mean'``, ``'max'`` or ``None``.
        Default is ``'max'``.
    """

    def __init__(self,
                 X: np.array,Y: np.array,
                 **kwargs):
        """Load dataset from npz."""
        super().__init__(**kwargs)

        self._x=torch.Tensor(X)
        self._y=torch.Tensor(Y)



    def _load_npz(self, dataset_path, labels_path):
        # Load dataset as csv
        dataset = np.load(dataset_path)

        # Load labels, can be found through csv or challenge description
        with open(labels_path, "r") as stream_json:
            self.labels = json.load(stream_json)

        R = dataset['R'].astype(np.float32)
        X = dataset['X'].astype(np.float32)
        Z = dataset['Z'].astype(np.float32)

        m = Z.shape[0]  # Number of training example
        K = Z.shape[1]  # Time serie length

        R = np.tile(R[:, np.newaxis, :], (1, K, 1))

        # Store R, Z and X as x and y
        #self._x = np.concatenate([Z, R], axis=-1)
        self._x = Z
        self._y = X

        # Normalize
        if self._normalize == "mean":
            mean = np.mean(self._x, axis=(0, 1))
            std = np.std(self._x, axis=(0, 1))
            self._x = (self._x - mean) / (std + np.finfo(float).eps)

            self._mean = np.mean(self._y, axis=(0, 1))
            self._std = np.std(self._y, axis=(0, 1))
            self._y = (self._y - self._mean) / (self._std + np.finfo(float).eps)
        elif self._normalize == "max":
            M = np.max(self._x, axis=(0, 1))
            m = np.min(self._x, axis=(0, 1))
            self._x = (self._x - m) / (M - m + np.finfo(float).eps)

            self._M = np.max(self._y, axis=(0, 1))
            self._m = np.min(self._y, axis=(0, 1))
            self._y = (self._y - self._m) / (self._M - self._m + np.finfo(float).eps)
        elif self._normalize is None:
            pass
        else:
            raise(
                NameError(f'Normalize method "{self._normalize}" not understood.'))

        # Convert to float32
        self._x = torch.Tensor(self._x)
        self._y = torch.Tensor(self._y)

    def rescale(self,
                y: np.ndarray,
                idx_label: int) -> torch.Tensor:
        """Rescale output from initial normalization.

        Arguments
        ---------
        y:
            Array to resize, of shape (K,).
        idx_label:
            Index of the output label.
        """
        if self._normalize == "max":
            return y * (self._M[idx_label] - self._m[idx_label] + np.finfo(float).eps) + self._m[idx_label]
        elif self._normalize == "mean":
            return y * (self._std[idx_label] + np.finfo(float).eps) + self._mean[idx_label]
        else:
            raise(
                NameError(f'Normalize method "{self._normalize}" not understood.'))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return (self._x[idx], self._y[idx])

    def __len__(self):
        return self._x.shape[0]



In [5]:
lobdata=LOBDataset(x,y)


dataset_train, dataset_test = random_split(lobdata, (180, 15))

In [6]:
dataloader_train = DataLoader(dataset_train,
                              batch_size=BATCH_SIZE,
                              shuffle=True
                             )
dataloader_test = DataLoader(dataset_test,
                             batch_size=BATCH_SIZE,
                             shuffle=False
                            )

In [7]:
# Load transformer with Adam optimizer and MSE loss function
net = Transformer(d_input, d_model, d_output, q, v, h, N, attention_size=attention_size, dropout=dropout, chunk_mode=chunk_mode, pe=pe).to(device)
optimizer = optim.Adam(net.parameters(), lr=LR)
loss_function = OZELoss(alpha=0.3)

NameError: name 'Transformer' is not defined